In [1]:
!export CUDA_VISIBLE_DEVICES=0


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Now import your GPU-related libraries, such as PyTorch
import torch
torch.cuda.empty_cache()

torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [23]:
# Import libraries and set up CUDA
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
torch.cuda.empty_cache()

import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler
import random

random_seed = 777  # You can change this value to any seed you prefer
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)


# Load and preprocess your data for H1, H2, and H3
model1_data = pd.read_csv("model1_2636.csv")
texts_h1 = model1_data['text'].tolist()
labels_h1 = model1_data.iloc[:, 1].values.tolist()

model2_data = pd.read_csv("model2_2636.csv")
texts_h2 = model2_data['text'].tolist()
labels_h2 = model2_data.iloc[:, 1].values.tolist()

model3_data = pd.read_csv("model3_2636.csv")
texts_h3 = model3_data['text'].tolist()
labels_h3 = model3_data.iloc[:, 1:].values.tolist()

In [3]:
# Import libraries and set up CUDA
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
torch.cuda.empty_cache()

import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler
import random

random_seed = 777  # You can change this value to any seed you prefer
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)


# Load and preprocess your data for H1, H2, and H3
model1_data = pd.read_csv("model1_2636.csv")
texts_h1 = model1_data['text'].tolist()
labels_h1 = model1_data.iloc[:, 1].values.tolist()

model2_data = pd.read_csv("model2_2636.csv")
texts_h2 = model2_data['text'].tolist()
labels_h2 = model2_data.iloc[:, 1].values.tolist()

model3_data = pd.read_csv("model3_2636.csv")
texts_h3 = model3_data['text'].tolist()
labels_h3 = model3_data.iloc[:, 1:].values.tolist()

#print("Total Records in H1:", len(texts_h1),"texts_h1", "with", len(labels_h1), "labels")
#print("Total Records in H2:", len(texts_h2),"texts_h2","with", len(labels_h2), "labels")
#print("Total Records in H3:", len(texts_h3),"texts_h3", "with", len(labels_h3), "labels")

# Split the data for each hierarchy
def split_data(texts, labels, train_ratio, val_ratio, test_ratio):
    total_samples = len(texts)
    train_size = int(total_samples * train_ratio)
    val_size = int(total_samples * val_ratio)
    test_size = total_samples - train_size - val_size  # Remaining samples for test

    train_texts = texts[:train_size]
    val_texts = texts[train_size:train_size + val_size]
    test_texts = texts[train_size + val_size:]

    train_labels = labels[:train_size]
    val_labels = labels[train_size:train_size + val_size]
    test_labels = labels[train_size + val_size:]

    return train_texts, val_texts, test_texts, train_labels, val_labels, test_labels

# Split the data for each hierarchy
train_texts_h1, val_texts_h1, test_texts_h1, train_labels_h1, val_labels_h1, test_labels_h1 = split_data(texts_h1, labels_h1, 0.7, 0.1, 0.2)
train_texts_h2, val_texts_h2, test_texts_h2, train_labels_h2, val_labels_h2, test_labels_h2 = split_data(texts_h2, labels_h2, 0.7, 0.1, 0.2)
train_texts_h3, val_texts_h3, test_texts_h3, train_labels_h3, val_labels_h3, test_labels_h3 = split_data(texts_h3, labels_h3, 0.7, 0.1, 0.2)

#print("Train Val Test Splitting:")
#print("H1:", len(train_texts_h1), len(val_texts_h1), len(test_texts_h1))
#print("H2:", len(train_texts_h2), len(val_texts_h2), len(test_texts_h2))
#print("H3:", len(train_texts_h3), len(val_texts_h3), len(test_texts_h3))

# Tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Define the TextDataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels_h1, labels_h2, labels_h3, tokenizer, max_length=512):
        self.texts = texts
        self.labels_h1 = labels_h1  # Now expecting a list of binary values for H1
        self.labels_h2 = np.array(labels_h2).reshape(-1, 1)  # Reshape labels_h2 to [batch_size, 1]
        self.labels_h3 = labels_h3  # Multi-label but binary values for H3
        self.tokenizer = tokenizer
        self.max_length = max_length

    # Rest of the code remains unchanged

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text, None, add_special_tokens=True, max_length=self.max_length,
            padding='max_length', return_token_type_ids=True, truncation=True
        )
        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'labels_h1': torch.tensor(self.labels_h1[idx], dtype=torch.float).unsqueeze(-1),
            'labels_h2': torch.tensor(self.labels_h2[idx], dtype=torch.float),
            'labels_h3': torch.tensor(self.labels_h3[idx], dtype=torch.float)
        }

# Create datasets and dataloaders for H1, H2, and H3
dataset_h1 = TextDataset(train_texts_h1 + val_texts_h1 + test_texts_h1, train_labels_h1 + val_labels_h1 + test_labels_h1, train_labels_h2 + val_labels_h2 + test_labels_h2, train_labels_h3 + val_labels_h3 + test_labels_h3, tokenizer)
dataset_h2 = TextDataset(train_texts_h2 + val_texts_h2 + test_texts_h2, train_labels_h1 + val_labels_h1 + test_labels_h1, train_labels_h2 + val_labels_h2 + test_labels_h2, train_labels_h3 + val_labels_h3 + test_labels_h3, tokenizer)
dataset_h3 = TextDataset(train_texts_h3 + val_texts_h3 + test_texts_h3, train_labels_h1 + val_labels_h1 + test_labels_h1, train_labels_h2 + val_labels_h2 + test_labels_h2, train_labels_h3 + val_labels_h3 + test_labels_h3, tokenizer)

# Split the datasets into train, val, and test for H1, H2, and H3
train_size_h1 = len(train_texts_h1)
val_size_h1 = len(val_texts_h1)
train_size_h2 = len(train_texts_h2)
val_size_h2 = len(val_texts_h2)
train_size_h3 = len(train_texts_h3)
val_size_h3 = len(val_texts_h3)

train_dataset_h1, val_dataset_h1, test_dataset_h1 = random_split(dataset_h1, [train_size_h1, val_size_h1, len(test_texts_h1)])
train_dataset_h2, val_dataset_h2, test_dataset_h2 = random_split(dataset_h2, [train_size_h2, val_size_h2, len(test_texts_h2)])
train_dataset_h3, val_dataset_h3, test_dataset_h3 = random_split(dataset_h3, [train_size_h3, val_size_h3, len(test_texts_h3)])

# Create dataloaders for H1, H2, and H3
train_dataloader_h1 = DataLoader(train_dataset_h1, batch_size=8, shuffle=True)
val_dataloader_h1 = DataLoader(val_dataset_h1,batch_size=8, shuffle=False)  # You can set shuffle to True if you want to shuffle the validation data.
train_dataloader_h2 = DataLoader(train_dataset_h2, batch_size=8, shuffle=True)
val_dataloader_h2 = DataLoader(val_dataset_h2, batch_size=8, shuffle=False)

train_dataloader_h3 = DataLoader(train_dataset_h3, batch_size=8, shuffle=True)
val_dataloader_h3 = DataLoader(val_dataset_h3, batch_size=8, shuffle=False)

# Define the model architecture for H1, H2, and H3
class HierarchicalClassifier(nn.Module):
    def __init__(self, num_labels_h3):
        super(HierarchicalClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(0.1)
        self.fc_h1 = nn.Linear(self.roberta.config.hidden_size, 1)  # Binary output for H1
        self.fc_h2 = nn.Linear(self.roberta.config.hidden_size, 1)  # Binary output for H2
        self.fc_h3 = nn.Linear(self.roberta.config.hidden_size, num_labels_h3)  # Multi-label output for H3

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids, attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        logits_h1 = self.fc_h1(self.dropout(pooled_output))
        logits_h2 = self.fc_h2(self.dropout(pooled_output))
        logits_h3 = self.fc_h3(self.dropout(pooled_output))
        return logits_h1, logits_h2, logits_h3

# Initialize and move the model to the appropriate device (CPU/GPU)
model = HierarchicalClassifier(num_labels_h3=len(train_labels_h3[0]))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(), lr=1e-5)  # You can adjust the learning rate as needed

# Training loop for H1, H2, and H3
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for batch in tqdm(dataloader, total=len(dataloader), desc="Training"):
        input_ids = batch['ids'].to(device)
        attention_mask = batch['mask'].to(device)
        labels_h1 = batch['labels_h1'].to(device)
        labels_h2 = batch['labels_h2'].to(device)
        labels_h3 = batch['labels_h3'].to(device)

        optimizer.zero_grad()

        logits_h1, logits_h2, logits_h3 = model(input_ids, attention_mask)
        loss_h1 = criterion(logits_h1, labels_h1)
        loss_h2 = criterion(logits_h2, labels_h2)
        loss_h3 = criterion(logits_h3, labels_h3)

        loss = loss_h1 + loss_h2 + loss_h3
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    return total_loss / len(dataloader)

# Validation loop for H1, H2, and H3
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_preds_h1, all_preds_h2, all_preds_h3 = [], [], []
    all_labels_h1, all_labels_h2, all_labels_h3 = [], [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, total=len(dataloader), desc="Validation"):
            input_ids = batch['ids'].to(device)
            attention_mask = batch['mask'].to(device)
            labels_h1 = batch['labels_h1'].to(device)
            labels_h2 = batch['labels_h2'].to(device)
            labels_h3 = batch['labels_h3'].to(device)

            logits_h1, logits_h2, logits_h3 = model(input_ids, attention_mask)
            loss_h1 = criterion(logits_h1, labels_h1)
            loss_h2 = criterion(logits_h2, labels_h2)
            loss_h3 = criterion(logits_h3, labels_h3)

            loss = loss_h1 + loss_h2 + loss_h3
            total_loss += loss.item()

            preds_h1 = torch.sigmoid(logits_h1)
            preds_h2 = torch.sigmoid(logits_h2)
            preds_h3 = torch.sigmoid(logits_h3)

            all_preds_h1.extend(preds_h1.cpu().numpy())
            all_preds_h2.extend(preds_h2.cpu().numpy())
            all_preds_h3.extend(preds_h3.cpu().numpy())

            all_labels_h1.extend(labels_h1.cpu().numpy())
            all_labels_h2.extend(labels_h2.cpu().numpy())
            all_labels_h3.extend(labels_h3.cpu().numpy())

    return total_loss / len(dataloader), all_preds_h1, all_preds_h2, all_preds_h3, all_labels_h1, all_labels_h2, all_labels_h3

# Training and evaluation for H1, H2, and H3
# Custom accuracy function
def custom_accuracy(y_true, y_pred):
    correct_labels = np.sum(np.equal(y_true, y_pred), axis=1)
    total_labels = y_true.shape[1]
    sample_accuracy = np.mean(correct_labels / total_labels)
    return sample_accuracy

def custom_precision_recall_f1(y_true, y_pred):
    # Calculate precision and recall for each sample
    sample_precisions = []
    sample_recalls = []
    for true, pred in zip(y_true, y_pred):
        true_positive = np.sum((true == 1) & (pred == 1))
        false_positive = np.sum((true == 0) & (pred == 1))
        false_negative = np.sum((true == 1) & (pred == 0))
        
        sample_precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
        sample_recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
        
        sample_precisions.append(sample_precision)
        sample_recalls.append(sample_recall)
    
    # Calculate average precision and recall across all samples
    avg_precision = np.mean(sample_precisions)
    avg_recall = np.mean(sample_recalls)
    
    # Calculate F1 score
    if avg_precision + avg_recall > 0:
        avg_f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
    else:
        avg_f1 = 0
    
    return avg_precision, avg_recall, avg_f1


# Training and evaluation for H1, H2, and H3
num_epochs = 33 # You can adjust the number of epochs as needed

for epoch in range(num_epochs):
    # Training for H1
    train_loss_h1 = train(model, train_dataloader_h1, optimizer, criterion, device)
    # Training for H2
    train_loss_h2 = train(model, train_dataloader_h2, optimizer, criterion, device)
    # Training for H3
    train_loss_h3 = train(model, train_dataloader_h3, optimizer, criterion, device)

    # Validation for H1
    val_loss_h1, val_preds_h1, _, _, val_labels_h1, _, _ = evaluate(model, val_dataloader_h1, criterion, device)
    # Validation for H2
    val_loss_h2, _, val_preds_h2, _, _, val_labels_h2, _ = evaluate(model, val_dataloader_h2, criterion, device)
    # Validation for H3
    val_loss_h3, _, _, val_preds_h3, _, _, val_labels_h3 = evaluate(model, val_dataloader_h3, criterion, device)

    # Metrics calculation for H1
    threshold_h1 = 0.5
    val_preds_h1_binary = (np.array(val_preds_h1) > threshold_h1).astype(int)

    acc_h1 = accuracy_score(val_labels_h1, val_preds_h1_binary)
    precision_h1 = precision_score(val_labels_h1, val_preds_h1_binary, average='micro')
    recall_h1 = recall_score(val_labels_h1, val_preds_h1_binary, average='micro')
    f1_h1 = f1_score(val_labels_h1, val_preds_h1_binary, average='micro')


    # Metrics calculation for H2
    threshold_h2 = 0.5
    val_preds_h2_binary = (np.array(val_preds_h2) > threshold_h2).astype(int)

    acc_h2 = accuracy_score(val_labels_h2, val_preds_h2_binary)
    precision_h2 = precision_score(val_labels_h2, val_preds_h2_binary, average='micro')
    recall_h2 = recall_score(val_labels_h2, val_preds_h2_binary, average='micro')
    f1_h2 = f1_score(val_labels_h2, val_preds_h2_binary, average='micro')

# Convert predictions and labels to binary using the threshold

    # Metrics calculation for H3
    threshold_h3 = 0.5
    val_preds_h3_binary = (np.array(val_preds_h3) > threshold_h3).astype(int)


    # Custom accuracy calculation for H3
    acc_h3_custom = custom_accuracy(np.array(val_labels_h3), val_preds_h3_binary)
    # Calculate custom precision, recall, and F1 score for H3
    precision_h3, recall_h3, f1_score_h3 = custom_precision_recall_f1(np.array(val_labels_h3), val_preds_h3_binary)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"H1: Train Loss: {train_loss_h1:.4f}, Val Loss: {val_loss_h1:.4f}, Accuracy: {acc_h1:.4f}")
    print(f"Precision H1: {precision_h1:.4f}, Recall H1: {recall_h1:.4f}, F1 H1: {f1_h1:.4f}")

    print(f"H2: Train Loss: {train_loss_h2:.4f}, Val Loss: {val_loss_h2:.4f}, Accuracy: {acc_h2:.4f}")
    print(f"Precision H2: {precision_h2:.4f}, Recall H2: {recall_h2:.4f}, F1 H2: {f1_h2:.4f}")

    print(f"H3: Train Loss: {train_loss_h3:.4f}, Val Loss: {val_loss_h3:.4f}, Custom Accuracy: {acc_h3_custom:.4f}")
    print(f"Precision H3: {precision_h3},Recall H3:{recall_h3},F1 Score H3: {f1_score_h3}")

# Saving the model
torch.save(model.state_dict(), 'model.pth')

# Loading the model
#loaded_model = YourModelClass(*args, **kwargs)  # Instantiate your model
#loaded_model.load_state_dict(torch.load('model.pth'))
#loaded_model.eval()  # Set the model to evaluation mode



2024-02-12 21:37:59.294814: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/data2/users/mpagare/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Validation: 100%|██████████| 33/33 [00:10<00:00,  3.12it/s]


Epoch 1/33
H1: Train Loss: 1.7604, Val Loss: 1.4489, Accuracy: 0.9278
Precision H1: 0.9278, Recall H1: 0.9278, F1 H1: 0.9278
H2: Train Loss: 1.5636, Val Loss: 1.4847, Accuracy: 0.6692
Precision H2: 0.6692, Recall H2: 0.6692, F1 H2: 0.6692
H3: Train Loss: 1.5059, Val Loss: 1.4030, Custom Accuracy: 0.6663
Precision H3: 0.6584510229947493,Recall H3:0.5230416604941319,F1 Score H3: 0.5829867949825093


Validation: 100%|██████████| 33/33 [00:10<00:00,  3.14it/s]


Epoch 2/33
H1: Train Loss: 1.4822, Val Loss: 1.3191, Accuracy: 0.9430
Precision H1: 0.9430, Recall H1: 0.9430, F1 H1: 0.9430
H2: Train Loss: 1.4396, Val Loss: 1.3551, Accuracy: 0.7110
Precision H2: 0.7110, Recall H2: 0.7110, F1 H2: 0.7110
H3: Train Loss: 1.3876, Val Loss: 1.3909, Custom Accuracy: 0.6718
Precision H3: 0.6720532319391636,Recall H3:0.513523749170137,F1 Score H3: 0.5821896018669244


Validation: 100%|██████████| 33/33 [00:10<00:00,  3.14it/s]


...

Epoch 31/33
H1: Train Loss: 0.1450, Val Loss: 0.5083, Accuracy: 0.9848
Precision H1: 0.9848, Recall H1: 0.9848, F1 H1: 0.9848
H2: Train Loss: 0.1344, Val Loss: 0.5180, Accuracy: 0.9620
Precision H2: 0.9620, Recall H2: 0.9620, F1 H2: 0.9620
H3: Train Loss: 0.1273, Val Loss: 0.5529, Custom Accuracy: 0.9354
Precision H3: 0.9339577195090503,Recall H3:0.9096376584969741,F1 Score H3: 0.9216372781626816


Validation: 100%|██████████| 33/33 [00:10<00:00,  3.13it/s]


Epoch 32/33
H1: Train Loss: 0.1228, Val Loss: 0.4788, Accuracy: 0.9810
Precision H1: 0.9810, Recall H1: 0.9810, F1 H1: 0.9810
H2: Train Loss: 0.1156, Val Loss: 0.5532, Accuracy: 0.9506
Precision H2: 0.9506, Recall H2: 0.9506, F1 H2: 0.9506
H3: Train Loss: 0.1215, Val Loss: 0.4990, Custom Accuracy: 0.9380
Precision H3: 0.9407475076676597,Recall H3:0.90881959189944,F1 Score H3: 0.9245079740000215


Validation: 100%|██████████| 33/33 [00:10<00:00,  3.12it/s]


Epoch 33/33
H1: Train Loss: 0.1189, Val Loss: 0.5005, Accuracy: 0.9848
Precision H1: 0.9848, Recall H1: 0.9848, F1 H1: 0.9848
H2: Train Loss: 0.1930, Val Loss: 0.5258, Accuracy: 0.9430
Precision H2: 0.9430, Recall H2: 0.9430, F1 H2: 0.9430
H3: Train Loss: 0.1237, Val Loss: 0.5319, Custom Accuracy: 0.9415
Precision H3: 0.9407174679988367,Recall H3:0.9122554990425712,F1 Score H3: 0.9262678932629732


In [4]:
# Create test datasets for H1, H2, and H3
test_dataset_h1 = TextDataset(test_texts_h1, test_labels_h1, test_labels_h2, test_labels_h3, tokenizer)
test_dataset_h2 = TextDataset(test_texts_h2, test_labels_h1, test_labels_h2, test_labels_h3, tokenizer)
test_dataset_h3 = TextDataset(test_texts_h3, test_labels_h1, test_labels_h2, test_labels_h3, tokenizer)

# Create test dataloaders for H1, H2, and H3
test_dataloader_h1 = DataLoader(test_dataset_h1, batch_size=8, shuffle=False)
test_dataloader_h2 = DataLoader(test_dataset_h2, batch_size=8, shuffle=False)
test_dataloader_h3 = DataLoader(test_dataset_h3, batch_size=8, shuffle=False)
# Testing loop for H1, H2, and H3
def test(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_preds_h1, all_preds_h2, all_preds_h3 = [], [], []
    all_labels_h1, all_labels_h2, all_labels_h3 = [], [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, total=len(dataloader), desc="Testing"):
            input_ids = batch['ids'].to(device)
            attention_mask = batch['mask'].to(device)
            labels_h1 = batch['labels_h1'].to(device)  # No need to reshape labels_h1
            labels_h2 = batch['labels_h2'].to(device)  # No need to reshape labels_h2

            #labels_h1 = batch['labels_h1'].unsqueeze(1).to(device)  # Reshape labels_h1
            #labels_h2 = batch['labels_h2'].unsqueeze(1).to(device)  # Reshape labels_h2
            #labels_h3 = batch['labels_h3'].unsqueeze(1).to(device)  # Reshape labels_h3
            labels_h3 = batch['labels_h3'].to(device)  # No need to reshape labels_h3
            print("Labels H1 shape:", labels_h1.shape)
            print("Labels H2 shape:", labels_h2.shape)
            print("Labels H3 shape:", labels_h3.shape)

            logits_h1, logits_h2, logits_h3 = model(input_ids, attention_mask)
            
            print("Logits H1 shape:", logits_h1.shape)
            print("Logits H2 shape:", logits_h2.shape)
            print("Logits H3 shape:", logits_h3.shape)
            
            
            
            loss_h1 = criterion(logits_h1, labels_h1)
            loss_h2 = criterion(logits_h2, labels_h2)
            loss_h3 = criterion(logits_h3, labels_h3)

            loss = loss_h1 + loss_h2 + loss_h3
            total_loss += loss.item()

            preds_h1 = torch.sigmoid(logits_h1)
            preds_h2 = torch.sigmoid(logits_h2)
            preds_h3 = torch.sigmoid(logits_h3)

            all_preds_h1.extend(preds_h1.cpu().numpy())
            all_preds_h2.extend(preds_h2.cpu().numpy())
            all_preds_h3.extend(preds_h3.cpu().numpy())

            all_labels_h1.extend(labels_h1.cpu().numpy())
            all_labels_h2.extend(labels_h2.cpu().numpy())
            all_labels_h3.extend(labels_h3.cpu().numpy())

    return total_loss / len(dataloader), all_preds_h1, all_preds_h2, all_preds_h3, all_labels_h1, all_labels_h2, all_labels_h3

# Testing for H1
test_loss_h1, test_preds_h1, _, _, test_labels_h1, _, _ = test(model, test_dataloader_h1, criterion, device)
# Testing for H2
test_loss_h2, _, test_preds_h2, _, _, test_labels_h2, _ = test(model, test_dataloader_h2, criterion, device)
# Testing for H3
test_loss_h3, _, _, test_preds_h3, _, _, test_labels_h3 = test(model, test_dataloader_h3, criterion, device)

# Metrics calculation for H1
threshold_h1 = 0.5
test_preds_h1_binary = (np.array(test_preds_h1) > threshold_h1).astype(int)

acc_h1 = accuracy_score(test_labels_h1, test_preds_h1_binary)
precision_h1 = precision_score(test_labels_h1, test_preds_h1_binary, average='micro')
recall_h1 = recall_score(test_labels_h1, test_preds_h1_binary, average='micro')
f1_h1 = f1_score(test_labels_h1, test_preds_h1_binary, average='micro')

# Metrics calculation for H2
threshold_h2 = 0.5
test_preds_h2_binary = (np.array(test_preds_h2) > threshold_h2).astype(int)

acc_h2 = accuracy_score(test_labels_h2, test_preds_h2_binary)
precision_h2 = precision_score(test_labels_h2, test_preds_h2_binary, average='micro')
recall_h2 = recall_score(test_labels_h2, test_preds_h2_binary, average='micro')
f1_h2 = f1_score(test_labels_h2, test_preds_h2_binary, average='micro')

# Metrics calculation for H3
threshold_h3 = 0.5
test_preds_h3_binary = (np.array(test_preds_h3) > threshold_h3).astype(int)

# Custom accuracy calculation for H3
acc_h3_custom = custom_accuracy(np.array(test_labels_h3), test_preds_h3_binary)
# Calculate custom precision, recall, and F1 score for H3
precision_h3, recall_h3, f1_score_h3 = custom_precision_recall_f1(np.array(test_labels_h3), test_preds_h3_binary)

print("Testing Results:")
print(f"H1: Test Loss: {test_loss_h1:.4f}, Accuracy: {acc_h1:.4f}")
print(f"Precision H1: {precision_h1:.4f}, Recall H1: {recall_h1:.4f}, F1 H1: {f1_h1:.4f}")

print(f"H2: Test Loss: {test_loss_h2:.4f}, Accuracy: {acc_h2:.4f}")
print(f"Precision H2: {precision_h2:.4f}, Recall H2: {recall_h2:.4f}, F1 H2: {f1_h2:.4f}")

print(f"H3: Test Loss: {test_loss_h3:.4f}, Custom Accuracy: {acc_h3_custom:.4f}")
print(f"Precision H3: {precision_h3}, Recall H3:{recall_h3}, F1 Score H3: {f1_score_h3}")




Testing:   0%|          | 0/66 [00:00<?, ?it/s]

Labels H1 shape: torch.Size([8, 1])
Labels H2 shape: torch.Size([8, 1])
Labels H3 shape: torch.Size([8, 13])
Logits H1 shape: torch.Size([8, 1])
Logits H2 shape: torch.Size([8, 1])
Logits H3 shape: torch.Size([8, 13])


Testing:   2%|▏         | 1/66 [00:00<00:23,  2.80it/s]

Labels H1 shape: torch.Size([8, 1])
Labels H2 shape: torch.Size([8, 1])
Labels H3 shape: torch.Size([8, 13])
Logits H1 shape: torch.Size([8, 1])
Logits H2 shape: torch.Size([8, 1])
Logits H3 shape: torch.Size([8, 13])


Testing:   3%|▎         | 2/66 [00:00<00:20,  3.07it/s]

...

Labels H1 shape: torch.Size([8, 1])
Labels H2 shape: torch.Size([8, 1])
Labels H3 shape: torch.Size([8, 13])
Logits H1 shape: torch.Size([8, 1])
Logits H2 shape: torch.Size([8, 1])
Logits H3 shape: torch.Size([8, 13])


Testing:  98%|█████████▊| 65/66 [00:21<00:00,  3.04it/s]

Labels H1 shape: torch.Size([8, 1])
Labels H2 shape: torch.Size([8, 1])
Labels H3 shape: torch.Size([8, 13])
Logits H1 shape: torch.Size([8, 1])
Logits H2 shape: torch.Size([8, 1])
Logits H3 shape: torch.Size([8, 13])


Testing: 100%|██████████| 66/66 [00:21<00:00,  3.04it/s]

Testing Results:
H1: Test Loss: 0.1025, Accuracy: 0.9981
Precision H1: 0.9981, Recall H1: 0.9981, F1 H1: 0.9981
H2: Test Loss: 0.1025, Accuracy: 0.9924
Precision H2: 0.9924, Recall H2: 0.9924, F1 H2: 0.9924
H3: Test Loss: 0.1025, Custom Accuracy: 0.9862
Precision H3: 0.9906166644803007, Recall H3:0.9820489117364117, F1 Score H3: 0.9863141822209809


In [9]:
#NEW UNSEEN

In [31]:
# Assuming the new unseen datasets are similar to 'model1_2636.csv', 'model2_2636.csv', 'model3_2636.csv'
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import numpy as np
import torch

import pandas as pd

# Loading and preprocessing new_data_h1
new_data_h1 = pd.read_csv("112_H_100_O_NEWMODEL1.CSV")
new_data_h1['text'].fillna("", inplace=True)  # Replace NaN values with empty strings
new_texts_h1 = new_data_h1['text'].tolist()
new_labels_h1 = new_data_h1.iloc[:, 1].values.tolist()

# Loading and preprocessing new_data_h2
new_data_h2 = pd.read_csv("112_H_100_O_NEWMODEL2.CSV")
new_data_h2['text'].fillna("", inplace=True)  # Replace NaN values with empty strings
new_texts_h2 = new_data_h2['text'].tolist()
new_labels_h2 = new_data_h2.iloc[:, 1].values.tolist()

# Loading and preprocessing new_data_h3 for multi-label classification
new_data_h3 = pd.read_csv("112_H_100_O_NEWMODEL3.CSV")
new_data_h3['text'].fillna("", inplace=True)  # Replace NaN values with empty strings

# Replace NaN values in label columns with 0 for new_data_h3
# Assuming all label columns are binary, fill NaN values with 0
for label_col in new_data_h3.columns[1:]:  # Skip the text column, only fill NaN in label columns
    new_data_h3[label_col].fillna(0, inplace=True)

new_texts_h3 = new_data_h3['text'].tolist()
new_labels_h3 = new_data_h3.iloc[:, 1:].values.tolist()  # Extract labels after filling NaN values




class TextDataset(Dataset):
    def __init__(self, texts, labels_h1, labels_h2, labels_h3, tokenizer, max_length=512):
        self.texts = texts
        self.labels_h1 = labels_h1  # Now expecting a list of binary values for H1
        self.labels_h2 = np.array(labels_h2).reshape(-1, 1)  # Reshape labels_h2 to [batch_size, 1]
        self.labels_h3 = labels_h3  # Multi-label but binary values for H3
        self.tokenizer = tokenizer
        self.max_length = max_length

    # Rest of the code remains unchanged
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text, None, add_special_tokens=True, max_length=self.max_length,
            padding='max_length', return_token_type_ids=True, truncation=True
        )
        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'labels_h1': torch.tensor(self.labels_h1[idx], dtype=torch.float).unsqueeze(-1),
            'labels_h2': torch.tensor(self.labels_h2[idx], dtype=torch.float),
            'labels_h3': torch.tensor(self.labels_h3[idx], dtype=torch.float)
        }
        
class HierarchicalClassifier(nn.Module):
    def __init__(self, num_labels_h3):
        super(HierarchicalClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(0.1)
        self.fc_h1 = nn.Linear(self.roberta.config.hidden_size, 1)  # Binary output for H1
        self.fc_h2 = nn.Linear(self.roberta.config.hidden_size, 1)  # Binary output for H2
        self.fc_h3 = nn.Linear(self.roberta.config.hidden_size, num_labels_h3)  # Multi-label output for H3

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids, attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        logits_h1 = self.fc_h1(self.dropout(pooled_output))
        logits_h2 = self.fc_h2(self.dropout(pooled_output))
        logits_h3 = self.fc_h3(self.dropout(pooled_output))
        return logits_h1, logits_h2, logits_h3

# Initialize and move the model to the appropriate device (CPU/GPU)
#model = HierarchicalClassifier(num_labels_h3=len(train_labels_h3[0]))

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()

from transformers import RobertaTokenizer

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Preprocess the texts and labels in the same way as the original datasets
# Assuming the preprocessing steps are included in the TextDataset class
# Create TextDataset instances for the new data
new_dataset_h1 = TextDataset(new_texts_h1, new_labels_h1, new_labels_h2, new_labels_h3, tokenizer)
new_dataset_h2 = TextDataset(new_texts_h2, new_labels_h1, new_labels_h2, new_labels_h3, tokenizer)
new_dataset_h3 = TextDataset(new_texts_h3, new_labels_h1, new_labels_h2, new_labels_h3, tokenizer)

# Create DataLoader instances for the new datasets
#new_dataloader_h1 = DataLoader(new_dataset_h1, batch_size=8, shuffle=False)
#new_dataloader_h2 = DataLoader(new_dataset_h2, batch_size=8, shuffle=False)
#new_dataloader_h3 = DataLoader(new_dataset_h3, batch_size=8, shuffle=False)


new_dataloader_h1 = DataLoader(new_dataset_h1, batch_size=8, shuffle=False, drop_last=True)
new_dataloader_h2 = DataLoader(new_dataset_h2, batch_size=8, shuffle=False, drop_last=True)
new_dataloader_h3 = DataLoader(new_dataset_h3, batch_size=8, shuffle=False, drop_last=True)

# Load the saved model
model_path = 'model.pth'  # Path to your saved model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = HierarchicalClassifier(num_labels_h3=len(new_labels_h3[0]))  # Make sure to provide the correct number of labels for H3
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode
model.to(device)  # Move the model to the appropriate device
def test(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_preds_h1, all_preds_h2, all_preds_h3 = [], [], []
    all_labels_h1, all_labels_h2, all_labels_h3 = [], [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, total=len(dataloader), desc="Testing"):
            input_ids = batch['ids'].to(device)
            attention_mask = batch['mask'].to(device)
            
            # Ensuring labels are correctly shaped and moved to the device
            labels_h1 = batch['labels_h1'].to(device)  # Shape: [batch_size, 1]
            labels_h2 = batch['labels_h2'].to(device)  # Shape: [batch_size, 1]
            labels_h3 = batch['labels_h3'].to(device)  # Shape: [batch_size, num_labels_h3]
            
            logits_h1, logits_h2, logits_h3 = model(input_ids, attention_mask)
            
            # Compute loss for each hierarchy
            loss_h1 = criterion(logits_h1, labels_h1)
            loss_h2 = criterion(logits_h2, labels_h2)
            loss_h3 = criterion(logits_h3, labels_h3)

            loss = loss_h1 + loss_h2 + loss_h3
            total_loss += loss.item()

            # Convert logits to probabilities for binary classification/multi-label classification
            preds_h1 = torch.sigmoid(logits_h1)
            preds_h2 = torch.sigmoid(logits_h2)
            preds_h3 = torch.sigmoid(logits_h3)

            # Store predictions and labels for metric calculation
            all_preds_h1.extend(preds_h1.cpu().numpy())
            all_preds_h2.extend(preds_h2.cpu().numpy())
            all_preds_h3.extend(preds_h3.cpu().numpy())

            all_labels_h1.extend(labels_h1.cpu().numpy())
            all_labels_h2.extend(labels_h2.cpu().numpy())
            all_labels_h3.extend(labels_h3.cpu().numpy())

    return total_loss / len(dataloader), all_preds_h1, all_preds_h2, all_preds_h3, all_labels_h1, all_labels_h2, all_labels_h3







# Testing the model on the new unseen data
test_loss_h1, test_preds_h1, _, _, test_labels_h1, _, _ = test(model, new_dataloader_h1, criterion, device)
test_loss_h2, _, test_preds_h2, _, _, test_labels_h2, _ = test(model, new_dataloader_h2, criterion, device)
test_loss_h3, _, _, test_preds_h3, _, _, test_labels_h3 = test(model, new_dataloader_h3, criterion, device)

# Calculate and print the metrics for H1, H2, and H3 using the same approach as before
# Use the appropriate threshold and custom functions for metrics calculation

# Metrics calculation for H1
threshold_h1 = 0.5
test_preds_h1_binary = (np.array(test_preds_h1) > threshold_h1).astype(int)

acc_h1 = accuracy_score(test_labels_h1, test_preds_h1_binary)
precision_h1 = precision_score(test_labels_h1, test_preds_h1_binary, average='micro')
recall_h1 = recall_score(test_labels_h1, test_preds_h1_binary, average='micro')
f1_h1 = f1_score(test_labels_h1, test_preds_h1_binary, average='micro')

# Metrics calculation for H2
threshold_h2 = 0.5
test_preds_h2_binary = (np.array(test_preds_h2) > threshold_h2).astype(int)

acc_h2 = accuracy_score(test_labels_h2, test_preds_h2_binary)
precision_h2 = precision_score(test_labels_h2, test_preds_h2_binary, average='micro')
recall_h2 = recall_score(test_labels_h2, test_preds_h2_binary, average='micro')
f1_h2 = f1_score(test_labels_h2, test_preds_h2_binary, average='micro')

# Metrics calculation for H3
threshold_h3 = 0.5
test_preds_h3_binary = (np.array(test_preds_h3) > threshold_h3).astype(int)

import numpy as np

def custom_accuracy(y_true, y_pred):
    correct_predictions = np.equal(y_true, y_pred)
    sample_accuracy = np.sum(correct_predictions, axis=1) / y_true.shape[1]
    return np.mean(sample_accuracy)

def custom_precision_recall_f1(y_true, y_pred):
    true_positives = np.sum((y_true == 1) & (y_pred == 1), axis=1)
    false_positives = np.sum((y_true == 0) & (y_pred == 1), axis=1)
    false_negatives = np.sum((y_true == 1) & (y_pred == 0), axis=1)

    precision = np.mean(true_positives / (true_positives + false_positives + 1e-8))
    recall = np.mean(true_positives / (true_positives + false_negatives + 1e-8))
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)

    return precision, recall, f1




# Custom accuracy calculation for H3
acc_h3_custom = custom_accuracy(np.array(test_labels_h3), test_preds_h3_binary)
# Calculate custom precision, recall, and F1 score for H3
precision_h3, recall_h3, f1_score_h3 = custom_precision_recall_f1(np.array(test_labels_h3), test_preds_h3_binary)

print("Testing Results:")
print(f"H1: Test Loss: {test_loss_h1:.4f}, Accuracy: {acc_h1:.4f}")
print(f"Precision H1: {precision_h1:.4f}, Recall H1: {recall_h1:.4f}, F1 H1: {f1_h1:.4f}")

print(f"H2: Test Loss: {test_loss_h2:.4f}, Accuracy: {acc_h2:.4f}")
print(f"Precision H2: {precision_h2:.4f}, Recall H2: {recall_h2:.4f}, F1 H2: {f1_h2:.4f}")

print(f"H3: Test Loss: {test_loss_h3:.4f}, Custom Accuracy: {acc_h3_custom:.4f}")
print(f"Precision H3: {precision_h3}, Recall H3:{recall_h3}, F1 Score H3: {f1_score_h3}")





Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Testing: 100%|██████████| 26/26 [00:07<00:00,  3.32it/s]

Testing Results:
H1: Test Loss: 0.4606, Accuracy: 0.9904
Precision H1: 0.9904, Recall H1: 0.9904, F1 H1: 0.9904
H2: Test Loss: 0.4606, Accuracy: 0.9904
Precision H2: 0.9904, Recall H2: 0.9904, F1 H2: 0.9904
H3: Test Loss: 0.4606, Custom Accuracy: 0.8924
Precision H3: 0.7959153672914849, Recall H3:0.8958478993148093, F1 Score H3: 0.8429301187466134
